# My version of shared notes

In [ ]:
!pip install line_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import cProfile

# Load the Excel file
file_path = "clinics.xls"
df = pd.read_excel(file_path)

df.head()

,bizID,bizCat,bizCatSub,bizName,bizAddr,bizCity,bizState,bizZip,bizPhone,bizFax,...,bizURL,locAreaCode,locFIPS,locTimeZone,locDST,locLat,locLong,locMSA,locPMSA,locCounty
0,1,Clinics,Clinics,Hino Ronald H MD,98-151 Pali Momi Street Suite 142,Aiea,HI,96701,(808)487-2477,NaN,...,NaN,808,15003,PST-2,N,21.3980,-157.8981,3320.0,NaN,Honolulu
1,2,Clinics,Clinics,Farmer Joesph F Md,1225 Breckenridge Drive,Little Rock,AR,72205,(501)225-2594,NaN,...,NaN,501,5119,CST,Y,34.7495,-92.3533,4400.0,NaN,Pulaski
2,3,Clinics,Clinics & Medical Centers,Najjar Fadi Md,1155 West Linda Avenue Suite B,Hermiston,OR,97838,(541)289-1122,NaN,...,NaN,541,41059,PST,Y,45.8456,-119.2817,NaN,NaN,Umatilla
3,4,Clinics,Clinics & Medical Centers,Kittson Memorial Upper Level Nursing Home,1010 South Birch Avenue,Hallock,MN,56728,(218)843-2525,NaN,...,NaN,218,27069,CST,Y,48.7954,-97.0090,NaN,NaN,Kittson
4,5,Clinics,Clinics & Medical Centers,Thompson Robert B Md,100 North Eagle Creek Drive,Lexington,KY,40509,(859)258-4000,NaN,...,www.lexingtonclinic.com,859,21067,EST,Y,37.9935,-84.3712,4280.0,NaN,Fayette


In [ ]:
df.columns

Index(['bizID', 'bizCat', 'bizCatSub', 'bizName', 'bizAddr', 'bizCity',
       'bizState', 'bizZip', 'bizPhone', 'bizFax', 'bizEmail', 'bizURL',
       'locAreaCode', 'locFIPS', 'locTimeZone', 'locDST', 'locLat', 'locLong',
       'locMSA', 'locPMSA', 'locCounty'],
      dtype='object')

In [ ]:
df['locLat'].head()

,locLat
0,21.3980
1,34.7495
2,45.8456
3,48.7954
4,37.9935


In [ ]:
df['locLong'].head()

,locLong
0,-157.8981
1,-92.3533
2,-119.2817
3,-97.0090
4,-84.3712


In [ ]:
# Extract latitude and longitude
df = df[['locLat', 'locLong']].dropna()
coords = df.values  # Convert to NumPy array

# Haversine function in km (not miles for my Convenience)
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES * c
    return total_miles

# Taking first log and lats as refrence
print(coords[0][1])
print(coords[0][0])

-157.8981
21.398


In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Base

In [ ]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(21.3980, -157.8981, df.iloc[i]['locLat'], df.iloc[i]['locLong'])
        distance_list.append(d)
    return distance_list
#cProfile.run("df['distance'] = haversine_looping(df)")

%timeit df['distance'] = haversine_looping(df)


2.33 ms ± 56 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Iterrows Haversine

In [ ]:
#%%% vectorize code by using series and iterrows
# Haversine applied on rows via iteration
%%timeit
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(21.3980, -157.8981, row['locLat'], row['locLong']))
#cProfile.run("df['distance'] = haversine_series")
df['distance'] = haversine_series

1.76 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Apply on rows

In [ ]:
#%%% Optimize further
# Timing apply on the Haversine function
%%timeit
df['distance'] = df.apply(lambda row: haversine(21.3980, -157.8981, row['locLat'], row['locLong']), \
                          axis=1)

%lprun -f haversine df.apply(lambda row: haversine(21.3980, -157.8981, row['locLat'], row['locLong']), axis=1)


7.01 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Vectorized implementation of Haversine applied on Pandas series

In [ ]:
%%timeit
# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(21.3980, -157.8981, df['locLat'], df['locLong'])
%lprun -f haversine haversine(21.3980, -157.8981,\
                              df['locLat'], df['locLong'])
# cProfile.run("df['distance'] = haversine(21.3980, -157.8981, df['locLat'], df['locLong'])")



6.1 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Vectorized implementation of Haversine applied on NumPy arrays

In [ ]:
%%timeit
# Vectorized implementation of Haversine applied on NumPy arrays
df['distance'] = haversine(21.3980, -157.8981, df['locLat'].values, df['locLong'].values)
%lprun -f haversine df['distance'] = haversine(21.3980, -157.8981,\
                        df['locLat'].values, df['locLong'].values)
# cProfile.run("df['distance'] = haversine(21.3980, -157.8981, df['locLat'].values, df['locLong'].values)")

3.09 ms ± 450 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Cythonized loop (unaltered)

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    mi = miles_constant * c
    return mi

In [ ]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(21.3980, -157.8981,\
                row['locLat'], row['locLong']), axis=1)

1.03 ms ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Redefined with data types and C libraries

In [ ]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad
    rad = 0.01745329252*deg
    return rad

cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef:
        float dlon
        float dlat
        float a
        float c
        float mi

    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    mi = 3959 * c
    return mi

In [ ]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(21.3980, -157.8981,\
                              row['locLat'], row['locLong']), axis=1)

656 µs ± 23.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
